# Data Loading

This script is needed to load the experiment data into the Sqlite database used. You can customize the input reports by modifying the `experiment_name` and `experiment_data_path` variables (by default, they will read from the output folder)

In [ ]:
import logging
import pandas as pd
from helpers.sqlite_helpers import pd_to_sqlDB, sql_query_to_pd, run_sql_query

experiment_name = "2025-05-30"
experiment_data_path = f"../executions/{experiment_name}/output/reports"

TOOLS = ('jdime', 'spork', 'last_merge', 'mergiraf')
TOOLS_TUPLES = [ ('jdime', 'last_merge'), ('spork', 'mergiraf'), ('last_merge', 'mergiraf') ]

## Creation of initial tables

### Loading data from tool executions on scenarios

In [ ]:
for tool in TOOLS:
    logging.info(f"Processing executions of {tool} tool")
    tool_executions_df = pd.read_csv(f'{experiment_data_path}/merge-tools/{tool}.csv', names=['project', 'merge_sha', 'file_path', 'output_file_path', 'result', 'time_in_ns'])
    tool_executions_df['scenario_id'] = tool_executions_df['project'] + ':' + tool_executions_df['merge_sha']
    pd_to_sqlDB(tool_executions_df, f"{tool}_executions")
    logging.info(f"Finished executions of {tool} tool")

### Equivalence Tool <-> Merge File

In [ ]:
tools_files = {
    'last_merge': 'merge_java-merge_last_merge_java.csv',
    'spork': 'merge_spork_normalized_java-merge_spork_spork_normalized_java.csv',
    'mergiraf': 'merge_java-merge_mergiraf_java.csv',
    'jdime': 'merge_jdime_normalized_java-merge_jdime_java.csv'
}

for tool in TOOLS:
    logging.info(f"Processing equivalency between of {tool} tool")
    tool_equivalency_df = pd.read_csv(f'{experiment_data_path}/syntactic-comparison/{tools_files[tool]}', names=['project', 'merge_sha', 'file_path', 'file_a', 'file_b', 'outputs_equivalent'])
    tool_equivalency_df['scenario_id'] = tool_equivalency_df['project'] + ':' + tool_equivalency_df['merge_sha']
    pd_to_sqlDB(tool_equivalency_df, f"{tool}_merge_equivalency")

### Equivalence Tool A <-> Tool B

In [ ]:
tools_files = {
    'jdime_last_merge': 'merge_jdime_java-merge_last_merge_jdime_normalized_java.csv',
    'spork_mergiraf': 'merge_mergiraf_spork_normalized_java-merge_spork_spork_normalized_java.csv',
    'last_merge_mergiraf': 'merge_mergiraf_format_normalized_java-merge_last_merge_format_normalized_java.csv'
}

for (tools, equivalency_file_name) in tools_files.items():
    tools_equivalency_df = pd.read_csv(f'{experiment_data_path}/syntactic-comparison/{equivalency_file_name}', names=['project', 'merge_sha', 'file_path', 'file_a', 'file_b', 'outputs_equivalent'])
    tools_equivalency_df['scenario_id'] = tools_equivalency_df['project'] + ':' + tools_equivalency_df['merge_sha']
    pd_to_sqlDB(tools_equivalency_df, f"{tools}_equivalency")

### Equivalence between conflicts Tool A <-> Tool B

In [ ]:
tools_files = {
    'jdime_last_merge': "merge_jdime_java-merge_last_merge_java.csv",
    'spork_mergiraf': "merge_mergiraf_java-merge_spork_java.csv",
    'last_merge_mergiraf': "merge_mergiraf_java-merge_last_merge_java.csv",
}

for (toolA, toolB) in TOOLS_TUPLES:
    tool_a_tool_b_file_name = tools_files[f"{toolA}_{toolB}"]
    tool_a_tool_b_conflicts_df = pd.read_csv(f'{experiment_data_path}/conflicts-comparison/{tool_a_tool_b_file_name}', names=['project', 'merge_sha', 'file_path', 'file_a_conflict', 'file_b_conflict', 'outputs_equivalent'])
    tool_a_tool_b_conflicts_df['scenario_id'] = tool_a_tool_b_conflicts_df['project'] + ':' + tool_a_tool_b_conflicts_df['merge_sha']
    pd_to_sqlDB(tool_a_tool_b_conflicts_df, f"{toolA}_{toolB}_conflicts")
    run_sql_query(f"""CREATE INDEX IF NOT EXISTS idx_file_{toolA}_{toolB}_conflicts ON {toolA}_{toolB}_conflicts (file_path, file_a_conflict, file_b_conflict, outputs_equivalent)""")

## Aggregations

### Global executions informations by file

In [ ]:
run_sql_query(f"""DROP VIEW IF EXISTS global_executions""")
run_sql_query(f"""
  CREATE VIEW global_executions AS SELECT
    spork_executions.scenario_id,
    spork_executions.project,
    spork_executions.merge_sha,
    spork_executions.file_path,
    spork_executions.result as spork_result,
    mergiraf_executions.result as mergiraf_result,
    jdime_executions.result as jdime_result,
    last_merge_executions.result as last_merge_result
  FROM
    spork_executions
  JOIN
    last_merge_executions
  ON
    spork_executions.scenario_id = last_merge_executions.scenario_id AND spork_executions.file_path = last_merge_executions.file_path
  JOIN
    mergiraf_executions
  ON
    spork_executions.scenario_id = mergiraf_executions.scenario_id AND spork_executions.file_path = mergiraf_executions.file_path
  JOIN
    jdime_executions
  ON
    spork_executions.scenario_id = jdime_executions.scenario_id AND spork_executions.file_path = jdime_executions.file_path
""")

sql_query_to_pd(f"""SELECT * FROM global_executions""")

### Information about execution per scenery on each tool

In [ ]:
def get_executions_per_commit_query(tool_name: str) -> str:
    return f"""
        SELECT
            e.scenario_id,
            e.project,
            e.merge_sha,
            SUM(time_in_ns) as time_in_ns,
            CASE 
                WHEN SUM(result = "TOOL_ERROR") > 0 THEN "TOOL_ERROR"
                WHEN SUM(result = "SUCCESS_WITH_CONFLICTS") > 0 THEN "SUCCESS_WITH_CONFLICTS"
                ELSE "SUCCESS_WITHOUT_CONFLICTS"
            END AS result,
            CASE WHEN SUM(outputs_equivalent) == COUNT(me.file_path) THEN 1 ELSE 0 END AS outputs_equivalent
        FROM
            {tool_name}_executions e
        JOIN
            {tool_name}_merge_equivalency me
        ON
            e.file_path = me.file_path
        GROUP BY
            e.scenario_id
    """

for tool in TOOLS:
    logging.info(f"Creating view {tool}_executions_per_commit")
    run_sql_query(f"""DROP VIEW IF EXISTS {tool}_executions_per_commit""")
    run_sql_query(f"""CREATE VIEW {tool}_executions_per_commit AS {get_executions_per_commit_query(tool)}""")

query = ' UNION '.join([f"SELECT scenario_id, time_in_ns, result, outputs_equivalent, '{tool}' as tool FROM {tool}_executions_per_commit" for tool in TOOLS])
sql_query_to_pd(query)

### Information about execution per scenery on each tool (excluding failures)

In [ ]:
def get_filtered_executions_per_commit_query(tool_name: str) -> str:
    return f"""
        SELECT
            *
        FROM
            {tool}_executions_per_commit
        WHERE
            result <> 'TOOL_ERROR'
    """

for tool in TOOLS:
    logging.info(f"Creating view {tool}_executions_per_commit_filtered")
    run_sql_query(f"""DROP VIEW IF EXISTS {tool}_executions_per_commit_filtered""")
    run_sql_query(f"""CREATE VIEW {tool}_executions_per_commit_filtered AS {get_filtered_executions_per_commit_query(tool)}""")

### Global information about execution in each scenery

In [ ]:
run_sql_query(f"""DROP VIEW IF EXISTS global_executions_per_commit""")
run_sql_query(f"""CREATE VIEW global_executions_per_commit AS SELECT
    spork.scenario_id,
    spork.project,
    spork.merge_sha,
    last_merge.result as last_merge_result,
    jdime.result as jdime_result,
    spork.result as spork_result,
    mergiraf.result as mergiraf_result
  FROM
    spork_executions_per_commit spork
  JOIN
    last_merge_executions_per_commit last_merge
  ON
    spork.scenario_id = last_merge.scenario_id
  JOIN
    mergiraf_executions_per_commit mergiraf
  ON
    spork.scenario_id = mergiraf.scenario_id
  JOIN
    jdime_executions_per_commit jdime
  ON
    spork.scenario_id = jdime.scenario_id
""")
sql_query_to_pd(f"""select * from global_executions_per_commit""")

### Global information about execution in each scenery (excluding failures)

In [ ]:
run_sql_query(f"""DROP VIEW IF EXISTS global_executions_per_commit_filtered""")
run_sql_query(f"""CREATE VIEW global_executions_per_commit_filtered AS SELECT
    spork.scenario_id,
    spork.project,
    spork.merge_sha,
    spork.result as spork_result,
    last_merge.result as last_merge_result,
    mergiraf.result as mergiraf_result,
    jdime.result as jdime_result
  FROM
    spork_executions_per_commit_filtered spork
  JOIN
    last_merge_executions_per_commit_filtered last_merge
  ON
    spork.scenario_id = last_merge.scenario_id
  JOIN
    mergiraf_executions_per_commit_filtered mergiraf
  ON
    spork.scenario_id = mergiraf.scenario_id
  JOIN
    jdime_executions_per_commit_filtered jdime
  ON
    spork.scenario_id = jdime.scenario_id
""")
sql_query_to_pd(f"""select * from global_executions_per_commit_filtered""")

### Syntactic equivalence between tools per scenario

In [ ]:
def get_output_equivalency_per_tool(toolA: str, toolB: str) -> str:
    return f"""
        SELECT
            scenario_id,
            project,
            merge_sha,
            CASE
                WHEN SUM(CASE WHEN outputs_equivalent = 1 THEN 1 ELSE 0 END) = COUNT(outputs_equivalent) THEN 1
                ELSE 0
            END as outputs_equivalent
        FROM
            {toolA}_{toolB}_equivalency
        GROUP BY
            scenario_id
    """

for (toolA, toolB) in TOOLS_TUPLES:
    run_sql_query(f"""DROP VIEW IF EXISTS {toolA}_{toolB}_equivalency_per_commit""")
    run_sql_query(f"""CREATE VIEW {toolA}_{toolB}_equivalency_per_commit AS {get_output_equivalency_per_tool(toolA, toolB)}""")
    display(sql_query_to_pd(f"""select * from {toolA}_{toolB}_equivalency_per_commit"""))

### Comparison between conflicts Tool A <-> Tool B

In [ ]:
for (tool_a, tool_b) in TOOLS_TUPLES:
  files_in_which_both_tools_found_conflict_and_they_are_equal_query = f"""
    SELECT
      DISTINCT scenario_id, file_path, 1 as all_conflicts_match
    FROM
      {tool_a}_{tool_b}_conflicts c1
    WHERE
      NOT EXISTS (
          SELECT 1
          FROM {tool_a}_{tool_b}_conflicts c2
          WHERE c1.file_path = c2.file_path
          AND c1.file_a_conflict = c2.file_a_conflict
          AND c1.file_b_conflict = c2.file_b_conflict
          AND c2.outputs_equivalent = 0
      )
  """

  files_in_which_both_tools_found_conflict_and_they_are_equal = sql_query_to_pd(files_in_which_both_tools_found_conflict_and_they_are_equal_query)
  file_paths_in_which_both_tools_found_conflict_and_they_are_equal = "', '".join(files_in_which_both_tools_found_conflict_and_they_are_equal['file_path'].tolist())

  files_in_which_both_tools_found_conflict_but_they_are_different_query = f"""
    SELECT
      scenario_id, file_path, 0 as all_conflicts_match
    FROM
      {tool_a}_{tool_b}_conflicts c1
    WHERE
      file_path NOT IN ('{file_paths_in_which_both_tools_found_conflict_and_they_are_equal}')
    GROUP BY
      file_path
  """

  tool_a_tool_b_global_conflicts_information_query = f"""
    SELECT
      scenario_id, file_path, 1 as all_conflicts_match
    FROM
      {tool_a}_{tool_b}_conflicts c1
    WHERE
      file_path IN ('{file_paths_in_which_both_tools_found_conflict_and_they_are_equal}')
    GROUP BY
      file_path
    UNION
    {files_in_which_both_tools_found_conflict_but_they_are_different_query}
  """

  tool_a_tool_b_global_conflicts_information_df = sql_query_to_pd(tool_a_tool_b_global_conflicts_information_query)
  pd_to_sqlDB(tool_a_tool_b_global_conflicts_information_df, f"{tool_a}_{tool_b}_global_conflicts_information")

In [ ]:
commits_df = pd.read_csv(f'{experiment_data_path}/commits.csv', names=['project', 'merge', 'right', 'left'])
commits_df['scenario_id'] = commits_df['project'] + ':' + commits_df['merge']
pd_to_sqlDB(commits_df, "commits")

In [ ]:
projects_df = pd.read_csv(f'{experiment_data_path}/projects.csv', names=['path', 'name'])
pd_to_sqlDB(projects_df, "projects")

### Retrieving data from GitHub Api of scenarios that require build and test execution

In these scenarios, tool A reports a conflict and tool B does not, however, B is not equivalent to the merge commit. In this case, we need to perform the analysis using the result of the test suite execution on B.

For running these scripts, we need to recover data regarding test executions with the GitHub API. For that, we need to provide a GitHub PAT --- you can reuse the same token used to execute the experiment.

In [ ]:
!pip install PyGithub

In [ ]:
GITHUB_TOKEN = ''
ANALYSIS_GITHUB_REPO_OWNER = 'jpedroh'
ANALYSIS_GITHUB_REPO_NAME = 'mining-framework-analysis'
WORKFLOW_ID = 'mining_framework.yaml'

from github import Github
from github import Auth

auth = Auth.Token(GITHUB_TOKEN)
g = Github(auth=auth)

def get_results_for_tool(tool: str, scenarios: pd.DataFrame):
    repository = g.get_repo(f"{ANALYSIS_GITHUB_REPO_OWNER}/{ANALYSIS_GITHUB_REPO_NAME}")
    result = []

    for scenario in scenarios.itertuples(index=False):
        project = scenario[1]
        merge_sha = scenario[2]
        branch = f"mining-framework-analysis_{project}_{merge_sha}_merge.{tool}.java"

        workflow = repository.get_workflow(WORKFLOW_ID)
        latest_workflow_run = workflow.get_runs(branch=branch, status='completed').get_page(0)[0]

        latest_workflow_jobs = latest_workflow_run.jobs().get_page(0)
        at_least_one_job_passes = any(job.conclusion == 'success' for job in latest_workflow_jobs)

        result.append({
            'scenario_id': scenario.scenario_id,
            'tool': tool,
            'status': 'success' if at_least_one_job_passes else 'failure'
        })

    return pd.DataFrame(result)
        
scenarios = sql_query_to_pd(f"""SELECT scenario_id, project, merge_sha FROM global_executions_per_commit_filtered""")

last_merge_results_df = get_results_for_tool('last_merge', scenarios)
jdime_results_df = get_results_for_tool('jdime', scenarios)

mergiraf_results_df = get_results_for_tool('mergiraf', scenarios)
spork_results_df = get_results_for_tool('spork', scenarios)

concatenated_results_df = pd.concat([last_merge_results_df, jdime_results_df, mergiraf_results_df, spork_results_df], ignore_index=True)
pd_to_sqlDB(concatenated_results_df, "test_results")